# 02 - Buscar datos (Search API)

Consultamos la **Search API** con topic, paginación, ordenación y query syntax. Con token **demo** puedes usar solo las queries `cats`, `dogs` o `cats AND dogs` (sin proyecto; quicksearch). Con token propio usa tu **project_id** y **topic_id**.

## 1. Configuración

In [ ]:
# @title Credenciales { display-mode: "form" }
ACCESS_TOKEN = "demo"  # @param {type:"string"}
PROJECT_ID = ""       # @param {type:"string"} (opcional con demo; para quicksearch deja vacío)
TOPIC_ID = ""         # @param {type:"string"} (opcional; si usas proyecto, pon un topic_id del notebook 01)

BASE_URL = "https://api.talkwalker.com"
import requests
import json

def tw_get(endpoint, params=None):
    p = {"access_token": ACCESS_TOKEN}
    if params:
        p.update(params)
    r = requests.get(f"{BASE_URL}{endpoint}", params=p)
    r.raise_for_status()
    return r.json()

def show(data):
    print(json.dumps(data, indent=2, ensure_ascii=False))

print("✅ Configuración lista.")

## 2. Búsqueda básica (quicksearch con demo)

Sin proyecto: usamos el endpoint de quicksearch con `q`. Con demo solo: `cats`, `dogs`, `cats AND dogs`.

In [ ]:
# Quicksearch (sin project_id): requiere q
if not PROJECT_ID:
    resp = tw_get("/api/v1/search/results", params={"q": "cats AND dogs", "hpp": 5})
else:
    params = {"hpp": 5}
    if TOPIC_ID:
        params["topic"] = TOPIC_ID
    resp = tw_get(f"/api/v1/search/p/{PROJECT_ID}/results", params=params)

if resp.get("status_code") != "0":
    print("Error:", resp.get("status_message"))
else:
    data = resp.get("result_content", {}).get("data", [])
    print(f"Total (aprox.): {resp.get('pagination', {}).get('total', 'N/A')}")
    print(f"Devueltos: {len(data)}")
    for i, item in enumerate(data[:3]):
        d = item.get("data", item)
        print(f"  {i+1}. {d.get('title', '')[:60]}...")

## 3. Paginación y orden por fecha

In [ ]:
params = {"hpp": 10, "offset": 0, "sort_by": "published", "sort_order": "desc"}
if PROJECT_ID:
    if TOPIC_ID:
        params["topic"] = TOPIC_ID
    resp = tw_get(f"/api/v1/search/p/{PROJECT_ID}/results", params=params)
else:
    params["q"] = "cats"
    resp = tw_get("/api/v1/search/results", params=params)

if resp.get("status_code") == "0":
    pag = resp.get("pagination", {})
    print("Total:", pag.get("total"), "| Next:", pag.get("next", "—")[:80])
    data = resp.get("result_content", {}).get("data", [])
    print(f"Resultados en esta página: {len(data)}")

## 4. Filtro con query syntax (ej. sentimiento negativo)

Solo tiene efecto con proyecto y datos que tengan sentimiento. Con demo puede no haber resultados.

In [ ]:
if PROJECT_ID and TOPIC_ID:
    resp = tw_get(f"/api/v1/search/p/{PROJECT_ID}/results", params={
        "topic": TOPIC_ID, "q": "sentiment:NEGATIVE", "hpp": 5
    })
    if resp.get("status_code") == "0":
        data = resp.get("result_content", {}).get("data", [])
        print(f"Menciones negativas: {len(data)} (muestra)")
else:
    print("Configura PROJECT_ID y TOPIC_ID para probar filtro por sentimiento.")

## 5. (Opcional) Exportar a pandas DataFrame

In [ ]:
# Descomenta si quieres usar pandas
# import pandas as pd
# def results_to_df(resp):
#     data = resp.get("result_content", {}).get("data", [])
#     rows = []
#     for item in data:
#         d = item.get("data", item)
#         rows.append({"title": d.get("title"), "url": d.get("url"), "published": d.get("published"), "sentiment": d.get("sentiment")})
#     return pd.DataFrame(rows)
# df = results_to_df(resp)
# df.head()